In [2]:
# remove warning message
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# required library
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from local_utils import detect_lp
from os.path import splitext,basename
from keras.models import model_from_json
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.mobilenet_v2 import preprocess_input
from sklearn.preprocessing import LabelEncoder
import glob

In [3]:
os.environ['PATH'] += os.pathsep + r'../../AIoT_SDK_5/aiot_sdk/tools/ai_tools/tflite2xcore/'

In [4]:
import os
import sys
sys.path.insert(0, os.path.abspath('../../AIoT_SDK_5/aiot_sdk/tools/ai_tools/tflite2xcore/'))

In [9]:
#Andrew Step 1
from tflite2xcore import utils, analyze, version

In [10]:
# Load model architecture, weight and labels
json_file = open('MobileNets_character_recognition.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights("License_character_recognition_weight.h5")
print("[INFO] Model loaded successfully...")

labels = LabelEncoder()
labels.classes_ = np.load('license_character_classes.npy')
print("[INFO] Labels loaded successfully...")

[INFO] Model loaded successfully...
[INFO] Labels loaded successfully...


In [11]:
#set up for tflite conversion
import logging
logging.getLogger("tensorflow").setLevel(logging.DEBUG)

import tensorflow as tf
import numpy as np
assert float(tf.__version__[:3]) >= 2.3


In [12]:
# ignore warning 
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from keras.preprocessing.image import ImageDataGenerator
from keras.applications import MobileNetV2
from keras.layers import AveragePooling2D
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import model_from_json
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import glob
import numpy as np
from PIL import Image 
import PIL
# Import the necessary libraries 
from numpy import asarray 
  

In [13]:
# Arange input data and corresponding labels
dataset_paths = glob.glob("*/dataset_characters/0/*.jpg")
X=[]
labels=[]

for image_path in dataset_paths:
  label = image_path.split(os.path.sep)[-2]
  image=load_img(image_path,target_size=(80,80))
  image=img_to_array(image)

  X.append(image)
  labels.append(label)

X = np.array(X,dtype="float32")
labels = np.array(labels)

print("[INFO] Find {:d} images with {:d} classes".format(len(X),len(set(labels))))
print(X.shape)

[INFO] Find 928 images with 1 classes
(928, 80, 80, 3)


In [14]:
from tflite2xcore.utils import quantize_converter

In [15]:
#Andrew Step 2
from tflite2xcore import utils, analyze, version
converter = tf.lite.TFLiteConverter.from_keras_model(model) 
quantize_converter(converter, representative_data=X) 
tflite_model = converter.convert()

# Save the model.
with open('char_recognize_step2_xform.tflite', 'wb') as f:
  f.write(tflite_model)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /tmp/tmpn34e1__u/assets


In [19]:
#Andrew Step 3
import tflite2xcore.converter as xcore_converter
outputFile = os.path.normpath("/mnt/c/Users/Kyle/Desktop/Plate_Reader/Plate_detect_and_recognize/char_recognize_model_fully_xformed.out")
print(outputFile)
inputFile = os.path.normpath("/mnt/c/Users/Kyle/Desktop/Plate_Reader/Plate_detect_and_recognize/char_recognize_step2_xform.tflite")
print(inputFile)
xcore_converter.convert(inputFile, outputFile)


/mnt/c/Users/Kyle/Desktop/Plate_Reader/Plate_detect_and_recognize/char_recognize_model_fully_xformed.out
/mnt/c/Users/Kyle/Desktop/Plate_Reader/Plate_detect_and_recognize/char_recognize_step2_xform.tflite


In [17]:
#visualize model
#tflite_visualize.py "/mnt/c/Users/Kyle/Desktop/Plate_Reader/Plate_detect_and_recognize/test.out" -o "/mnt/c/Users/Kyle/Desktop/Plate_Reader/Plate_detect_and_recognize/test_visualize.html"